In [17]:
!pip install pandas seaborn scikit-learn mathplot numpy chromadb

In [2]:
import ollama
import chromadb
import numpy as np
import pandas as pd

# Movie dataset
courses = pd.read_csv("course_data.csv")

# Store in ChromaDB
client = chromadb.Client()
# client.delete_collection(name="courses")
collection = client.get_or_create_collection(name="courses")

# Generate embeddings
for i in courses.index:
    title = courses['course_title'][i]
    embedding = ollama.embeddings(model="nomic-embed-text", prompt=title)["embedding"]
    collection.add(
        ids=[str(courses["course_id"][i])],
        embeddings=[embedding],
        metadatas=[{"title": title}]
    )

print("Embedding created..")

Embedding created..


In [16]:
# Recommendation function
def recommend_courses(query, collection, top_k=2):
    query_embedding = ollama.embeddings(model="nomic-embed-text", prompt=query)["embedding"]
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)

    recommendations = results["metadatas"][0]
    distances = results["distances"][0]

    for course, distance in zip(recommendations, distances):
        print(f"{course['title']}, distance: {distance}")  # Convert distance to similarity
    print('-' * 30)
    return recommendations

# Test recommendation
query = "A futuristic world with advanced technology and rebellion"
# recommendations = recommend_courses(query, collection, 5)

recommend_courses("Angular Tutorial For Beginners", collection, 5)
recommend_courses("Angular", collection, 5)

recommend_courses("Frontend", collection, 5)


Angular Tutorial For Beginners, distance: 0.0
AngularJS For Beginners, distance: 110.56587219238281
AngularJs :basics for beginners, distance: 117.82424926757812
Angular 4 (2+) Crash Course for Beginners, distance: 124.49296569824219
AngularJS for Beginners, Single-Page Applications Made Easy, distance: 165.32266235351562
------------------------------
Angular Masterclass, distance: 234.79507446289062
Angular 2 From The Ground Up, distance: 241.49473571777344
Angular Material - Quickstart, distance: 245.57675170898438
Angular 4 Front To Back, distance: 256.9195556640625
Angular Tutorial For Beginners, distance: 265.4273681640625
------------------------------
Introduction to Frontend Web Development For  Beginners, distance: 223.35568237304688
Foundations of Front-End Web Development, distance: 262.5880126953125
Complete Guide to Front-End Web Development and Design, distance: 298.62359619140625
Parse Server: From Front End to Full Stack, distance: 339.77642822265625
JavaScript - Becom

[{'title': 'Introduction to Frontend Web Development For  Beginners'},
 {'title': 'Foundations of Front-End Web Development'},
 {'title': 'Complete Guide to Front-End Web Development and Design'},
 {'title': 'Parse Server: From Front End to Full Stack'},
 {'title': 'JavaScript - Become Higly Paid Front End Developer'}]